# 第 9 章： SCD Type 2 による履歴管理

データ分析基盤に蓄積されるデータは常に変化し続けています。企業の取引データや注文情報は日々更新され、その過程でテーブルの履歴情報は失われがちです。</br>
しかし、「過去のある時点でのデータがどうだったか」を知ることは、様々なビジネスシーンで重要な要素となります。例えば、ECサイトの注文データを分析する際、過去の注文履歴を正確に把握することは、顧客満足度や売上分析において不可欠です。  

通常、ソースデータをテーブルに取り込む際、データは最新の状態に更新されます。例えば、注文ステータスが変更された場合、データベース内の該当レコードは新しい値に上書きされます。このような設計では、過去のデータは失われてしまいます。この問題に対処するために、「Slowly Changing Dimensions（SCD）」という概念が発展してきました。特に SCD Type 2 は、データの変更履歴を保持するための効果的な手法として広く採用されています。  

このハンズオンでは、Apache Iceberg の変更ログビュー（Change Log View）機能を活用して、SCD Type 2 パターンを効率的に実装する方法を学びます。これにより、データの変更履歴を完全に保持し、時間軸に沿った精度の高いデータ分析が可能になります。

## SCD Type 2 の基礎知識

SCD Type 2 は、「Slowly Changing Dimensions（緩やかに変化するディメンション）」の一種で、データの変更履歴を保持するための効果的な手法です。このモデルでは、ソースデータのレコードが変更された場合に、既存レコードを上書きする代わりに、変更前のレコードを履歴として保持したまま新しいレコードを追加します。

各レコードには以下のような時間軸に関する属性が追加されます：

1. **有効開始日（effective_start_date）**：そのレコードが有効になった日時
2. **有効終了日（effective_end_date）**：そのレコードが無効になった（次のバージョンに置き換えられた）日時
3. **現行フラグ（current_flag）**：そのレコードが現在有効かどうかを示すフラグ

例えば、注文ステータスと金額の変更をSCD Type 2で表現すると次のようになります：

| order_id | customer_id | amount | status | order_date | effective_start_date | effective_end_date | current_flag |
|----------|-------------|--------|--------|------------|----------------------|---------------------|------------|
| ORD-1001 | CUST-101 | 12000 | Pending | 2024-05-01 | 2024-05-01 09:15:00 | 2024-05-01 14:30:00 | FALSE |
| ORD-1001 | CUST-101 | 10800 | Pending | 2024-05-01 | 2024-05-01 14:30:00 | 2024-05-02 10:45:00 | FALSE |
| ORD-1001 | CUST-101 | 15000 | Processing | 2024-05-01 | 2024-05-02 10:45:00 | 2024-05-03 16:20:00 | FALSE |
| ORD-1001 | CUST-101 | 15000 | Cancelled | 2024-05-01 | 2024-05-03 16:20:00 | NULL | TRUE |

このモデルにより、「過去の任意の時点でのデータ状態を正確に再現する」ことが可能になります。

## Apache Iceberg による SCD Type 2 の実装

Apache Iceberg では、変更ログビュー（Change Log View）を活用することで、SCD Type 2 を効率的に実装できます。変更ログビューは、Iceberg テーブルに対するすべての変更操作（挿入、更新、削除）の履歴を自動的に記録し、クエリ可能なビューとして提供する機能です。

このアプローチでは、ソースデータの変更を取り込む際は、対象の Iceberg テーブルに対して通常の更新操作を行うだけで済みます。テーブルの作成時点で SCD Type 2 専用のスキーマを設計する必要はなく、標準的なデータ操作のみでテーブルを管理できます。

### 事前準備

## Spark　セッションの初期化
まず、Spark　セッションを初期化します。

In [ ]:
import pyspark
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
import pandas as pd
CATALOG = "my_catalog"
CATALOG_URL = "http://server:8181/"
S3_ENDPOINT = "http://minio:9000"
SPARK_VERSION = pyspark.__version__
SPARK_MINOR_VERSION = '.'.join(SPARK_VERSION.split('.')[:2])
ICEBERG_VERSION = "1.7.1"

In [ ]:
spark = (
    SparkSession.builder
        .config("spark.jars.packages", f"org.apache.iceberg:iceberg-spark-runtime-{SPARK_MINOR_VERSION}_2.12:{ICEBERG_VERSION},org.apache.iceberg:iceberg-aws-bundle:{ICEBERG_VERSION}")
        .config(f"spark.sql.catalog.{CATALOG}", "org.apache.iceberg.spark.SparkCatalog")
        .config(f"spark.sql.catalog.{CATALOG}.type", "rest")
        .config(f"spark.sql.catalog.{CATALOG}.uri", CATALOG_URL)
        .config(f"spark.sql.catalog.{CATALOG}.s3.endpoint", S3_ENDPOINT)
        .config(f"spark.sql.catalog.{CATALOG}.view-endpoints-supported", "true")
        .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
        .config("spark.sql.defaultCatalog", "my_catalog")
        .getOrCreate()
)

In [ ]:
%sql spark

In [ ]:
%%sql
CREATE DATABASE IF NOT EXISTS scdtype2 --  ハンズオン用データベースを作成

## 注文テーブルの作成

次に、ECサイトの注文データを想定したサンプルデータを使用して注文テーブルを作成します。

In [ ]:
from pyspark.sql import Row
from datetime import datetime

# 現在のタイムスタンプを取得
current_time = datetime.now()

# 注文データセットの作成
orders_data = [
    {"order_id": "ORD-1001", "customer_id": "CUST-101", "amount": 12000, "status": "Pending", "order_date": "2024-05-01", "shipping_fee": 0, "discount": 0},
    {"order_id": "ORD-1002", "customer_id": "CUST-102", "amount": 8500, "status": "Pending", "order_date": "2024-05-01", "shipping_fee": 500, "discount": 0},
    {"order_id": "ORD-1003", "customer_id": "CUST-103", "amount": 23000, "status": "Pending", "order_date": "2024-05-02", "shipping_fee": 0, "discount": 2000},
    {"order_id": "ORD-1004", "customer_id": "CUST-101", "amount": 15700, "status": "Pending", "order_date": "2024-05-02", "shipping_fee": 700, "discount": 0},
    {"order_id": "ORD-1005", "customer_id": "CUST-104", "amount": 7200, "status": "Pending", "order_date": "2024-05-03", "shipping_fee": 0, "discount": 800}
]

# DataFrameの作成
df_orders = spark.createDataFrame([Row(**x) for x in orders_data])

# Icebergテーブルの作成 - 一時ビューを使わずに直接DataFrameからテーブルを作成
spark.sql("DROP TABLE IF EXISTS scdtype2.orders PURGE")
df_orders.write.format("iceberg").mode("overwrite").saveAsTable("scdtype2.orders")

# テーブルの確認
spark.sql("SELECT * FROM scdtype2.orders").show()

## テーブルデータの更新（変更履歴の作成）
注文テーブルを更新し、変更履歴を作成します。例として、ここでは以下の変更を行うことにします。

1. 注文ORD-1001に割引を適用し、金額を変更
2. 注文のステータスを「Processing」に変更し、配送料を追加
3. 注文のキャンセル処理
4. 他の注文ORD-1002, ORD-1004の更新
5. 新規注文ORD-1006の追加

最初に、注文ORD-1001に割引を適用し、金額を変更します。この更新により、注文ORD-1001の金額が12000円から10800円に変更され、1200円の割引が記録されます。

In [ ]:
# 注文ORD-1001に割引を適用（金額変更）
spark.sql("""
UPDATE scdtype2.orders
SET amount = 10800, discount = 1200
WHERE order_id = 'ORD-1001'
""")

次に、注文の処理が進み、配送オプションが追加されたため、金額とステータスを更新します。この更新では、配送料4200円が追加されて合計金額が15000円となり、ステータスが「Processing」に変更されています。

In [ ]:
spark.sql("""
UPDATE scdtype2.orders
SET amount = 15000, status = 'Processing', shipping_fee = 4200
WHERE order_id = 'ORD-1001'
""")

顧客からのキャンセル要求を受けて、注文ステータスを更新します。

In [ ]:
spark.sql("""
UPDATE scdtype2.orders
SET status = 'Cancelled'
WHERE order_id = 'ORD-1001'
""")

ORD-1001以外の注文についても、いくつかの更新を行います。

In [ ]:
spark.sql("""
UPDATE scdtype2.orders
SET status = 'Processing'
WHERE order_id = 'ORD-1002'
""")

In [ ]:
# 注文ORD-1004の配送オプションを変更（配送料変更）
spark.sql("""
UPDATE scdtype2.orders
SET shipping_fee = 1500, amount = 16500
WHERE order_id = 'ORD-1004'
""")

最後に、新しい注文を追加します。

In [ ]:
spark.sql("""
INSERT INTO scdtype2.orders VALUES
('ORD-1006', 'CUST-105', 9800, 'Pending', '2024-05-03', 0, 1200)
""")

In [ ]:
# 更新後のテーブルを確認
spark.sql("SELECT * FROM scdtype2.orders").show()

この時点では、テーブルには最新の状態のみが表示されており、過去の変更履歴は見えません。しかし、Icebergは内部的にすべての変更を記録しています。

## 変更ログビューの作成

Iceberg の プロシージャを使用して、変更ログビュー（Change Log View）を作成します。これにより、テーブルに対する全ての変更操作（INSERT, UPDATE, DELETE）の履歴が記録されたビューが得られます。

In [ ]:
# 変更ログビューの作成
spark.sql("""
CALL system.create_changelog_view('scdtype2.orders', 'orders_changelog')
""")

## 変更履歴の確認

作成した変更ログビューを使用して、注文テーブルの変更履歴を確認します。

In [ ]:
# 変更ログビューの確認
spark.sql("""
SELECT *
FROM orders_changelog
ORDER BY order_id, _change_ordinal, _change_type
""").show(truncate=False)

特定の注文の履歴だけを確認することもできます：

In [ ]:
# 注文ORD-1001の変更履歴を確認
spark.sql("""
SELECT order_id, status, amount, shipping_fee, discount, _change_type, _change_ordinal
FROM orders_changelog
WHERE order_id = 'ORD-1001'
ORDER BY _change_ordinal, _change_type
""").show()

変更ログビューでは、注文ORD-1001の状態がどのように変化したかを時系列で確認できます：
1. 初期注文：Pending状態で12000円
2. 割引適用：金額が10800円に変更（1200円の割引）
3. 配送オプション追加とステータス変更：金額が15000円に増加、ステータスがProcessingに変更、配送料が4200円追加
4. キャンセル処理：ステータスがCancelledに変更

## SCD Type 2のデータ構造を持った一時テーブルの作成

変更ログビューとスナップショット情報を結合して、SCD Type 2のデータ構造を持った一時テーブルを作成します。ここでは、注文ORD-1001のレコードを対象にします。

このクエリでは、以下の処理を行っています：

1. 変更ログビューとスナップショット情報を結合して、各変更のコミット時間を取得
2. 各レコードの有効開始日をコミット時間に設定
3. 各レコードの有効終了日を次の変更の有効開始日に設定（最新レコードの場合はnull）
4. 現行フラグを設定（最新レコードかつ削除されていない場合はtrue）

これにより、有効開始日、有効終了日、現行フラグを持つ履歴テーブルが生成されます。

In [ ]:
# 変更ログビューとスナップショット情報を結合して、SCD Type 2ビューを作成
spark.sql("""
WITH changelog_with_timestamps AS (
    SELECT
        cl.*,
        s.snapshot_id,
        s.committed_at,
        s.committed_at AS effective_start_date
    FROM orders_changelog cl
    JOIN scdtype2.orders.snapshots s
    ON cl._commit_snapshot_id = s.snapshot_id
)
SELECT
    order_id, 
    customer_id, 
    amount, 
    status, 
    order_date,
    shipping_fee,
    discount,
    effective_start_date,
    LEAD(effective_start_date) OVER (
        PARTITION BY order_id 
        ORDER BY effective_start_date
    ) AS effective_end_date,
    CASE WHEN LEAD(effective_start_date) OVER (
        PARTITION BY order_id 
        ORDER BY effective_start_date
    ) IS NULL AND _change_type != 'DELETE' THEN true
    ELSE false
    END AS current_flag
FROM changelog_with_timestamps
WHERE _change_type IN ('INSERT', 'UPDATE_AFTER')
ORDER BY order_id, effective_start_date
""").createOrReplaceTempView('orders_scd2')

# SCD Type 2ビューの確認
spark.sql("""
SELECT 
    order_id, 
    customer_id, 
    amount, 
    status, 
    shipping_fee,
    discount,
    effective_start_date, 
    effective_end_date, 
    current_flag
FROM orders_scd2
WHERE order_id = 'ORD-1001'
ORDER BY effective_start_date
""").show(truncate=False)

これにより、SCD Type 2の構造が自動的に生成されました。注文ORD-1001の変更履歴が時系列で明確に表示され、いつどのような変更があったかを簡単に追跡できます。

## SCD Type 2ビューの活用

SCD Type 2ビューを使用した分析例をいくつか紹介します。

### 特定状態のデータ確認

有効期間を使って特定ステータスの注文情報を確認する方法を示します。例えば、処理中（Processing）状態の注文のみを表示するクエリです：

In [ ]:
# Processing状態だった注文情報
spark.sql("""
SELECT 
    order_id, 
    customer_id, 
    amount, 
    status,
    shipping_fee, 
    discount
FROM orders_scd2
WHERE status = 'Processing'
ORDER BY order_id
""").show()

### 現在有効なデータのみの取得

現在有効な注文データのみを取得します。

In [ ]:
# 現在有効な注文データ
spark.sql("""
SELECT 
    order_id, 
    customer_id, 
    amount, 
    status,
    shipping_fee,
    discount
FROM orders_scd2
WHERE current_flag = true
ORDER BY order_id
""").show()

### 注文ステータスの変更回数分析

In [ ]:
# 注文ごとのステータス変更回数
spark.sql("""
SELECT 
    order_id,
    COUNT(*) - 1 AS status_change_count
FROM orders_scd2
GROUP BY order_id
ORDER BY status_change_count DESC
""").show()

### SCD Type 2ビューの永続化

必要に応じて、SCD Type 2ビューを永続化することも可能です。これにより、SCD Type 2形式のデータが永続的なテーブルとして保存され、後続の分析で使用できるようになります。特に定期的に履歴分析を行う場合や、複数のシステムから履歴データにアクセスする必要がある場合に有用です：

In [ ]:
# SCD Type 2ビューの永続化
spark.sql("""
CREATE TABLE IF NOT EXISTS scdtype2.orders_history
USING iceberg
AS SELECT * FROM orders_scd2
""")

In [ ]:
spark.sql("""
SELECT * FROM scdtype2.orders_history
""").show()

## SCD Type 2パターンのまとめ

このハンズオンでは、Apache Icebergの変更ログビュー機能を活用してSCD Type 2パターンを実装しました。このアプローチには以下のような利点があります：

1. **シンプルなデータ操作**: 通常のINSERT/UPDATE操作だけでデータを管理できます。複雑なETLプロセスを構築する必要がありません。
2. **自動的な変更履歴の記録**: Icebergが変更履歴を自動的に記録するため、変更追跡のための追加コードを書く必要がありません。
3. **スナップショット管理**: Icebergのスナップショット機能により、データの一貫性が保証されます。
4. **時間軸に沿った分析の容易さ**: 任意の時点でのデータの状態を簡単に再現できます。

SCD Type 2パターンは、コンプライアンス要件、監査対応、時系列分析などが必要な多くの業界で重要な役割を果たします。Icebergを使用することで、このパターンをより効率的かつ信頼性高く実装できるようになります。